In [1]:
import config
import geocoder
import pandas as pd

In [94]:
postal_codes_MID = pd.read_excel(r"C:\Users\william.gruintal\Desktop\coursera_projects\Coursera_Capstone_Final\CodigosPostalesMerida_20210517.xlsx")
headers = postal_codes_MID.iloc[0]
postal_codes_MID = postal_codes_MID.iloc[1:]
postal_codes_MID.columns = headers
postal_codes_MID.columns = ['CodigoPostal', 'Estado', 'Municipio', 'Ciudad', 'Tipo', 'Asentamiento', 'Oficina']
postal_codes_MID.drop(['Oficina'], axis = 1, inplace = True)
postal_codes_MID.reset_index(drop = True, inplace = True)
postal_codes_MID.head(10)

,CodigoPostal,Estado,Municipio,Ciudad,Tipo,Asentamiento
0,97000,Yucatán,Mérida,Mérida,Colonia,Mérida Centro
1,97000,Yucatán,Mérida,Mérida,Fraccionamiento,La Quinta
2,97000,Yucatán,Mérida,Mérida,Fraccionamiento,Los Cocos
3,97000,Yucatán,Mérida,Mérida,Fraccionamiento,Privada Del Maestro
4,97000,Yucatán,Mérida,Mérida,Colonia,Privada Garcia Gineres C - 29
5,97000,Yucatán,Mérida,Mérida,Fraccionamiento,Jardines de San Sebastian
6,97003,Yucatán,Mérida,Mérida,Fraccionamiento,Los Reyes
7,97050,Yucatán,Mérida,Mérida,Colonia,Alcalá Martín
8,97050,Yucatán,Mérida,Mérida,Colonia,Yucatán
9,97059,Yucatán,Mérida,Mérida,Fraccionamiento,Señorial


In [95]:
postal_codes_MID.describe()

,CodigoPostal,Estado,Municipio,Ciudad,Tipo,Asentamiento
count,632,632,632,513,632,632
unique,140,1,1,1,16,619
top,97203,Yucatán,Mérida,Mérida,Fraccionamiento,La Guadalupana
freq,35,632,632,513,383,2


In [96]:
postal_codes_MID.shape
#print('postal_codes_MID dataframe shape is: {} rows and {} columns'.format(postal_codes_MID.shape[0], postal_codes_MID.shape[1]))

(632, 6)

In [97]:
# check for null values
postal_codes_MID['CodigoPostal'].isnull().sum()

0

In [42]:
#testing
#postal_codes_MID = postal_codes_MID.iloc[0:10]
#postal_codes_MID

,CodigoPostal,Estado,Municipio,Ciudad,Tipo,Asentamiento
0,97000,Yucatán,Mérida,Mérida,Colonia,Mérida Centro
1,97000,Yucatán,Mérida,Mérida,Fraccionamiento,La Quinta
2,97000,Yucatán,Mérida,Mérida,Fraccionamiento,Los Cocos
3,97000,Yucatán,Mérida,Mérida,Fraccionamiento,Privada Del Maestro
4,97000,Yucatán,Mérida,Mérida,Colonia,Privada Garcia Gineres C - 29
5,97000,Yucatán,Mérida,Mérida,Fraccionamiento,Jardines de San Sebastian
6,97003,Yucatán,Mérida,Mérida,Fraccionamiento,Los Reyes
7,97050,Yucatán,Mérida,Mérida,Colonia,Alcalá Martín
8,97050,Yucatán,Mérida,Mérida,Colonia,Yucatán
9,97059,Yucatán,Mérida,Mérida,Fraccionamiento,Señorial


In [100]:
#postal_codes = [97179, 97000]
lat_long_coords = []
for i in postal_codes_MID['CodigoPostal']:
    lat_long1 = [i]
    lat_long2 = geocoder.arcgis('{}.Merida, YUC'.format(i))
    while(lat_long2.latlng is None):
        lat_long2 = geocoder.arcgis('{}, Toronto, Ontario'.format(i))
        print (i, lat_long2.latlng)
    lat_long1.extend(lat_long2.latlng)
    lat_long_coords.append(lat_long1)
    print(i, lat_long2.latlng)
print('coordinates loaded')



97000 [20.965910000000065, -89.62702981199999]
97000 [20.965910000000065, -89.62702981199999]
97000 [20.965910000000065, -89.62702981199999]
97000 [20.965910000000065, -89.62702981199999]
97000 [20.965910000000065, -89.62702981199999]
97000 [20.965910000000065, -89.62702981199999]
97003 [20.951190000000054, -89.64422999999994]
97050 [20.989415000000065, -89.62007598199995]
97050 [20.989415000000065, -89.62007598199995]
97059 [20.990435000000048, -89.61933733399997]
97060 [20.985573482000063, -89.64190999999994]
97068 [20.978545000000054, -89.64222124999998]
97069 [20.981115000000045, -89.63693879899995]
97070 [20.989240000000052, -89.63208340499995]
97070 [20.989240000000052, -89.63208340499995]
97070 [20.989240000000052, -89.63208340499995]
97080 [20.98337435800005, -89.61331499999994]
97088 [20.981350000000077, -89.61353441299997]
97089 [20.981381699000053, -89.61565999999999]
97098 [20.960215000000062, -89.60854108299998]
97099 [20.969535000000064, -89.60665391099997]
97100 [20.9927

97173 [20.952765000000056, -89.58735339699996]
97173 [20.952765000000056, -89.58735339699996]
97173 [20.952765000000056, -89.58735339699996]
97173 [20.952765000000056, -89.58735339699996]
97173 [20.952765000000056, -89.58735339699996]
97173 [20.952765000000056, -89.58735339699996]
97174 [20.945651003000023, -89.59658499999995]
97174 [20.945651003000023, -89.59658499999995]
97175 [20.964915000000076, -89.56691579899996]
97176 [20.95459500000004, -89.57513599999999]
97176 [20.95459500000004, -89.57513599999999]
97176 [20.95459500000004, -89.57513599999999]
97176 [20.95459500000004, -89.57513599999999]
97176 [20.95459500000004, -89.57513599999999]
97176 [20.95459500000004, -89.57513599999999]
97176 [20.95459500000004, -89.57513599999999]
97177 [20.952214966000042, -89.60247999999996]
97178 [20.95939500000003, -89.56553906799996]
97179 [20.954285000000027, -89.59580875999995]
97179 [20.954285000000027, -89.59580875999995]
97180 [20.951721511000073, -89.60845499999994]
97189 [20.94411334700

97220 [20.99241500000005, -89.65905262499996]
97225 [20.98354500000005, -89.64835595999995]
97225 [20.98354500000005, -89.64835595999995]
97226 [20.995195000000024, -89.66826721399997]
97227 [20.98829364000005, -89.65747999999996]
97227 [20.98829364000005, -89.65747999999996]
97229 [20.98950007700006, -89.65353499999998]
97229 [20.98950007700006, -89.65353499999998]
97229 [20.98950007700006, -89.65353499999998]
97229 [20.98950007700006, -89.65353499999998]
97229 [20.98950007700006, -89.65353499999998]
97229 [20.98950007700006, -89.65353499999998]
97229 [20.98950007700006, -89.65353499999998]
97229 [20.98950007700006, -89.65353499999998]
97230 [20.977500000000077, -89.64969564799998]
97230 [20.977500000000077, -89.64969564799998]
97237 [20.987919600000055, -89.66747499999997]
97238 [20.97832545800003, -89.65919999999994]
97238 [20.97832545800003, -89.65919999999994]
97238 [20.97832545800003, -89.65919999999994]
97238 [20.97832545800003, -89.65919999999994]
97238 [20.97832545800003, -89.

97302 [21.06121500000006, -89.62118338199997]
97302 [21.06121500000006, -89.62118338199997]
97302 [21.06121500000006, -89.62118338199997]
97302 [21.06121500000006, -89.62118338199997]
97302 [21.06121500000006, -89.62118338199997]
97302 [21.06121500000006, -89.62118338199997]
97302 [21.06121500000006, -89.62118338199997]
97303 [21.098375000000033, -89.70687111699993]
97303 [21.098375000000033, -89.70687111699993]
97303 [21.098375000000033, -89.70687111699993]
97303 [21.098375000000033, -89.70687111699993]
97304 [21.138133853000056, -89.64228999999995]
97304 [21.138133853000056, -89.64228999999995]
97305 [21.042245000000037, -89.56490893499995]
97305 [21.042245000000037, -89.56490893499995]
97305 [21.042245000000037, -89.56490893499995]
97305 [21.042245000000037, -89.56490893499995]
97305 [21.042245000000037, -89.56490893499995]
97305 [21.042245000000037, -89.56490893499995]
97305 [21.042245000000037, -89.56490893499995]
97305 [21.042245000000037, -89.56490893499995]
97305 [21.0422450000

In [101]:
columns = ['CodigoPostal', 'Latitud', 'Longitud']
lat_long_coords = pd.DataFrame(lat_long_coords)
lat_long_coords.columns = columns
lat_long_coords.head(10)

,CodigoPostal,Latitud,Longitud
0,97000,20.965910,-89.627030
1,97000,20.965910,-89.627030
2,97000,20.965910,-89.627030
3,97000,20.965910,-89.627030
4,97000,20.965910,-89.627030
5,97000,20.965910,-89.627030
6,97003,20.951190,-89.644230
7,97050,20.989415,-89.620076
8,97050,20.989415,-89.620076
9,97059,20.990435,-89.619337


In [102]:
lat_long_coords.shape

(632, 3)

In [103]:
print(postal_codes_MID.dtypes)
print(lat_long_coords.dtypes)

CodigoPostal    object
Estado          object
Municipio       object
Ciudad          object
Tipo            object
Asentamiento    object
dtype: object
CodigoPostal      int64
Latitud         float64
Longitud        float64
dtype: object


In [104]:
postal_codes_MID['CodigoPostal'] = postal_codes_MID['CodigoPostal'].astype('int64')
lat_long_coords['CodigoPostal'] = lat_long_coords['CodigoPostal'].astype('int64')

In [105]:
# merge dataframes
merida_neighborhoods = pd.merge(left = postal_codes_MID, right = lat_long_coords, left_index = True, right_index = True)#left_on = 'CodigoPostal', right_on = 'CodigoPostal')
merida_neighborhoods.drop('CodigoPostal_y', axis = 1, inplace = True)
merida_neighborhoods.rename(columns = {'CodigoPostal_x': 'CodigoPostal'}, inplace = True)
merida_neighborhoods.head(30)

,CodigoPostal,Estado,Municipio,Ciudad,Tipo,Asentamiento,Latitud,Longitud
0,97000,Yucatán,Mérida,Mérida,Colonia,Mérida Centro,20.965910,-89.627030
1,97000,Yucatán,Mérida,Mérida,Fraccionamiento,La Quinta,20.965910,-89.627030
2,97000,Yucatán,Mérida,Mérida,Fraccionamiento,Los Cocos,20.965910,-89.627030
3,97000,Yucatán,Mérida,Mérida,Fraccionamiento,Privada Del Maestro,20.965910,-89.627030
4,97000,Yucatán,Mérida,Mérida,Colonia,Privada Garcia Gineres C - 29,20.965910,-89.627030
5,97000,Yucatán,Mérida,Mérida,Fraccionamiento,Jardines de San Sebastian,20.965910,-89.627030
6,97003,Yucatán,Mérida,Mérida,Fraccionamiento,Los Reyes,20.951190,-89.644230
7,97050,Yucatán,Mérida,Mérida,Colonia,Alcalá Martín,20.989415,-89.620076
8,97050,Yucatán,Mérida,Mérida,Colonia,Yucatán,20.989415,-89.620076
9,97059,Yucatán,Mérida,Mérida,Fraccionamiento,Señorial,20.990435,-89.619337


In [106]:
merida_neighborhoods.shape

(632, 8)

In [107]:
g = geocoder.arcgis('Merida, YUC')
print('Mérida, Yucatán coordinates are: {}, {}.'.format(g.latlng[0], g.latlng[1]))

Mérida, Yucatán coordinates are: 20.96670000000006, -89.62316999999996.
